# Compute CVaR Efficiently

### Import Library

In [1]:
using Plots
using JuMP
import HiGHS
using DataFrames
using BenchmarkTools
using Optim

# Document Organization

In this document, I will explain what is CVaR. Several methods of computing CVaR and lead readers to understand what is the most efficient way to compute CVaR. 

###### 1 . Define distribution and precompute variables

a. explain the (distribution) function which create a distribution ($d$) dataframe/object from discrete random variable ($X$) and probability mass function ($p$).

b. Precompute some repeatedly used variable could be beneficial for speed and readability. We define $\Delta X$, Psum (CDF) and $X_{1:i}^TP_{1:i}$ for some function those required these precompute values. 

- Iteratively summing floating point variable could lead to high floating point arithmetic precision loss.
    - [x] Use a higher precision random variable if needed. (suggested)
    - [ ] Use the sum function repeatedly is slow $O(N^2)$ but more accurate.

###### 2 . Joint distribution VS Decomposition
a. We here define the Primal (CVaR and neat_CVaR) and the Decomposition (CVaR_Decom) representation of CVaR.

b. Then we will compare the computational efficiency of computing CVaR. We showed that compute CVaR from joint distribution is more efficient compared to via decomposition.
- [x] Joint
- [ ] Decomposition

###### 3 . Optimize CVaR
Last but not least we reorganized the function to compute the CVaR of a vector of **decending ordered risks** in linear time. 

- For Speed improvement (N $\alpha$s)
    - [x] Precompute values that required to solve repeatedly.
    - [x] Solving risk in decending order could reduce time complexity.


# 1 . Define Problem

In [2]:
α = 0.8

X_1 = [1,1,2,2]
X_2 = [50,60,70,80]

p1j = [0.1,0.2,0.5,0.2]
p2j = [0.5,0.3,0.1,0.1]

p_1 = 0.1
p_2 = 0.9

0.9

Given Conditional Distribution $X_1$ and $X_2$ and the conditional probability of $p_1$ and $p_2$. We can calculate the joint distribution $X$ as follow.

In [3]:
X = [X_1; X_2]
p = [p_1 * p1j; p_2 * p2j]

8-element Vector{Float64}:
 0.010000000000000002
 0.020000000000000004
 0.05
 0.020000000000000004
 0.45
 0.27
 0.09000000000000001
 0.09000000000000001

Let use a single DataFrame ($d$) instead of two Vectors ($X$ and $p$) to represent the distribution.

### distribution(X,p) 
- Combine values and pmf ($X$,$p$) into a single dataframe $d$.
- Remove values $x$ with zero probability of occuring.
- Aggegate distribution $d$ with value $X$ by adding the Prob $Pr$ of same values.
- Sort the distribution $d$ with values $X$ in increasing order.
- return $d$

In [4]:
function distribution(X,p)
    d = DataFrame(X = X, p = p)
    d = d[d.p .> 0,:]
    d = combine(groupby(d, ["X"]),df -> DataFrame(p = sum(df.p)) ) 
    sort!(d,["X"]) 
    return d
end
d_1 = distribution(X_1,p1j)
d_2 = distribution(X_2,p2j)
p_cond = [p_1;p_2] 
d = distribution(X,p)

Row,X,p
,Int64,Float64
1,1,0.03
2,2,0.07
3,50,0.45
4,60,0.27
5,70,0.09
6,80,0.09


#### Preliminary 1 
To compare the Asymptotic computational complexity between algorithms, we define $N$ as the number of values with non-zero probability that the discrete variable $X$ could take. We define $M$ as the total number of $\alpha$s required to compute given distribution of $X$.

# 2 . Joint vs Decomposition

### Define CVaR

For $\alpha \in (0,1)$, the definition of CVaR is 
$$\text{CVaR}_{\alpha}(X) = \frac{\int_\alpha^1 \text{VaR}_{\alpha'}(X) d\alpha'}{1-\alpha} = \frac{\int_0^{1-\alpha} Q_X(\lambda') d\lambda'}{1-\alpha} = \frac{\int_0^{1-\alpha} F^{-1}_X(\lambda') d\lambda'}{1-\alpha}$$
. Let $\lambda = 1-\alpha$ we can also write them as
$$\text{CVaR}_{\alpha}(X) = \frac{\int_\alpha^1 \text{VaR}_{\alpha'}(X) d\alpha'}{\lambda} = \frac{\int_0^{\lambda} Q_X(\lambda') d\lambda'}{\lambda} = \frac{\int_0^{\lambda} F^{-1}_X(\lambda') d\lambda'}{\lambda}$$
where $\text{VaR}_{\alpha'}(X)$ refers to the $\alpha'$ Value of Risk of a distribution $X$, and $Q_X(\lambda) = F^{-1}_X(\lambda)$ refers to the $\lambda$ inverse distribution function (quantile function) of X. Here we present the straight forward algorithm for $\text{CVaR}_{\alpha}(X) = \frac{\int_0^{\lambda} Q_X(\lambda') d\lambda'}{\lambda}$, all other primal method are in the spirit of this algorithm:

In [5]:
# CVaR function takes in a distribution d and a risk parameter alpha. 
function CVaR(d,α)
    # Set lambda
    λ = 1-α
    
    # Special Case:
    if λ == 0
        return minimum(d[d.p .> 0,:].X)
    end  
    
    # General Cases (Take Integral over 0 to lambda) Q_X(lambda') dlambda'
    Psum = 0
    for i in 1:nrow(d)
        if λ <= Psum + d.p[i]
            return (transpose(d.X[1:i])*[d.p[1:(i-1)];λ-Psum])/λ
        end
        Psum = Psum + d.p[i]
    end  
    return (transpose(d.X)*d.p)
end

# Solve CVaR for multiple Alphas.
function CVaR_Vec(d,Alpha)
    return [CVaR(d,α) for α in Alpha]
end

CVaR_Vec (generic function with 1 method)

#### Helper function (Pre-compute Variable)
Precompute $\Delta X$ takes **$O(N)$-time**. Assuming $X$ is $N$ elements sorted distribution with values $\{x_1,x_2,x_3,\cdots,x_{N}\}$, $\Delta X_1 = \min\{X\} = x_1$ and $\Delta X_i = x_i - x_{i-1}$. 

Precompute $\sum^{i}_{j=1}P(x_j) = F(x_i)$ Psum or CDF for $i \in 1\cdots N$ iteratively takes **$O(N)$-time**.

Precompute $\sum^{i}_{j=1}[ x_j \cdot P(x_j) ] =  X_{1:i}^T P_{1:i}$ for $i \in 1\cdots N$ iteratively takes **$O(N)$-time**.

<font color='green'>The precompute function below will precompute some repeatedly used values to speed up computation in some algorithms.</font>

##### <font color='red'> Minor Issue:</font>
- (cumulativeSum) iteratively adding Psum += i.Pr could lead to minor floating point arithmetic precision loss.
    - [x] use variable type of higher precision (suggested)
    - [ ] use accurateCumSum O(N^2) function

In [6]:
# This delta function is the inverse of cumsum
function delta(V)
    return [V[1];V[Not(1)]-V[Not(length(V))]]
end

delta(cumsum(1:10))'# test delta function

1×10 adjoint(::Vector{Int64}) with eltype Int64:
 1  2  3  4  5  6  7  8  9  10

### CVaR search
$$\lambda \text{CVaR}_{\alpha}(X) = \mathbb{E}[~X ~ 1_{\{X \leq x_\alpha\}}~] ~+~ x_\alpha(~\lambda -\mathbb{P}[X \leq x_\alpha]~) $$
where $x_\alpha = \text{VaR}_{\alpha}(X) = Q_{X}(\lambda)$.


In [7]:
# CVaR_Search function takes in a distribution (d) and a vector of risk (Alpha)
function search_CVaR(d,α)
    # Set lambda
    λ = 1-α
    if λ == 0
        return(minimum(d[d.p .> 0,:].X))
    end
    αi = min(searchsortedfirst(d.Psum,λ),length(d.Psum))
    return( ( d.XTP[αi] + d.X[αi] * (λ - d.Psum[αi]) ) / λ )
end

# Solve CVaR for multiple Alphas.
function search_CVaR_Vec(d,Alpha)
    # Here we precompute repeatedly used values, Psum and XTP.
    d.Psum = cumsum(d.p)
    d.XTP = cumsum(d.X .* d.p)
    
    return [search_CVaR(d,α) for α in Alpha]
end

search_CVaR_Vec (generic function with 1 method)

### Neat CVaR (Piecewise Linear)
Since $(1-\alpha)\cdot$CVaR$_\alpha(X)$ is piecewise linear, we can compute CVaR as a piecewise linear function for discrete random variable $X$ as 
    $$(1-\alpha) \text{CVaR}_\alpha(X) = \int_0^{\lambda} Q_X(\lambda') d\lambda' = \sum_{i=1}^{N} \big( \Delta X_i \cdot (1-\alpha - \sum^{i-1}_{j=1}P(x_j))_{+}\big)$$
    $$\text{CVaR}_\alpha(X) = \frac{\int_0^{\lambda} Q_X(\lambda') d\lambda'}{\lambda} =  \frac{\sum_{i=1}^{N} \big( \Delta X_i \cdot (\lambda - \sum^{i-1}_{j=1}P(x_j))_{+} \big)}{\lambda}$$
Assuming $X$ is $N$ element sorted distribution with values $\{x_1,x_2,x_3,\cdots,x_{N}\}$, $\Delta X_1 = \min\{X\}$ and $\Delta X_i = x_i - x_{i-1}$.

In [8]:
function neat_CVaR(d,α,λ=1-α) # O(N)
    if λ == 0
        return minimum(d[d.p .> 0,:].X)
    else
        return (transpose(d.ΔX)*max.(zeros(nrow(d)),λ .- d.Psum .+ d.p))/λ
    end
end

# CVaR method to solve for multiple Alphas
function neat_CVaR_Vec(d,Alpha)
    d.ΔX = delta(d.X)      # O(N)
    d.Psum = cumsum(d.p)   # O(N)
    return [neat_CVaR(d,α) for α in Alpha]
end

neat_CVaR_Vec (generic function with 1 method)

To solve for $M$ amount of different $\alpha$ risk level, the original method and the neat method is $O(NM)$, the neat method would perform faster in certain cercumstance because the factor for the $NM$ is smaller for neat method.

### CVaR via Decomposition
In this section we will intorudce the decomposition method by Pflug to solve the CVaR
$$\text{CVaR}_{\alpha}(X) = \inf_Z \{\mathbb{E}[Z \cdot \text{CVaR}_{1-(1-\alpha)Z}(X|f_i)] \mid \mathbb{E}[Z] = 1, 0 \leq Z \leq \frac{\mathbb{1}}{1-\alpha}  \}$$
For the example above with only two possible conditions, $\mathbb{E}[Z] = 1 \implies z_2  = \frac{1 - z_1 p_1}{p_2}$ and we will only require to minimize an univariate ($z_1$) function.

In [9]:
f(z_1,α) = p_1 * z_1 * neat_CVaR(d_1,1-(1-α)*z_1) + p_2 *((1-p_1*z_1)/p_2)* neat_CVaR(d_2,1-(1-α)*((1-p_1*z_1)/p_2))

function CVaR_Decom(α)
    if α == 1
        return min(minimum(d_1[d_1.p .> 0,:].X),minimum(d_2[d_2.p .> 0,:].X))
    else
        return Optim.minimum(optimize(z_1 -> f(z_1,α),0,1/(1-α),abs_tol=1e-12,rel_tol=1e-12)) 
    end
end

function CVaR_Decom_Vec(Alpha)
    d_1.ΔX = delta(d_1.X)      # O(N)
    d_1.Psum = cumsum(d_1.p)   # O(N)
    d_2.ΔX = delta(d_2.X)      # O(N)
    d_2.Psum = cumsum(d_2.p)   # O(N)
    return [CVaR_Decom(α) for α in Alpha]
end

CVaR_Decom_Vec (generic function with 1 method)

### Speed comparison

In [10]:
Alpha = LinRange(1,0,1001)

CVaR_ori = CVaR_Vec(d,Alpha)
t_ori = @elapsed CVaR_Vec(d,Alpha)

CVaR_neat = neat_CVaR_Vec(d,Alpha)
t_neat = @elapsed neat_CVaR_Vec(d,Alpha)

CVaR_search = search_CVaR_Vec(d,Alpha)
t_search = @elapsed search_CVaR_Vec(d,Alpha)

CVaR_decom = CVaR_Decom_Vec(Alpha)
t_decom = @elapsed CVaR_Decom_Vec(Alpha)

DiffNeat = DataFrame(Alg = ["ori";"search";"neat";"decom"],time = [maximum(abs.(CVaR_ori .- CVaR_neat));maximum(abs.(CVaR_search .- CVaR_neat)); 0;maximum(abs.(CVaR_decom .- CVaR_neat))])
TimeDf = DataFrame(Alg = ["ori";"search";"neat";"decom"],time = [t_ori;t_search; t_neat;t_decom])

Row,Alg,time
,String,Float64
1,ori,0.0014564
2,search,0.001073
3,neat,0.0042119
4,decom,0.766608


Both methods with joint distribution (neat and ori) outperform decomposition (decom) method by huge margin.

##### Conclusion : Compute CVaR with joint distribution is faster by huge margin compare to decomposition with LP.

# 3 . Optimizes CVaR for multiple $\alpha$

### Issue of CVaR
The CVaR algorithm above is genearlly good enough if we are solving for only one single $\alpha$. However, there are room for improvement to the algorithm when we intend to solve many $\alpha$s.

- For the algorithm above, the computation required to solve a $\alpha$ is $O(N)$ for $N = \text{nrow}(d)$. Therefore solving $M = |\alpha|$ many $\alpha$ would require $O(MN)$ time.
    - To reduce the computation time we could
        - [x] Precompute and store the values we would use repeatedly (eg: CDF (Psum) and $(X^T P)$).
        - [x] Solve $M$ risk level $\alpha$ in $O(M+N)$-time.

The Fast_CVaR algorithm will takes in a vector of **<font color='red'> sorted decending order</font>** $\alpha$s and calculate their CVaR all together. Instead of running $O(MN)$-time, it takes only $O(M+N)$-time to run. Assumed that the vector risk parameter is sorted, we will not need to seek for previous seen $i$ for the CVaR of smaller $\lambda = 1-\alpha$. Instead each $i$ only increment once from $1 \cdots N$ and each $j$ only increment once from $1 \cdots M$.

In [11]:
# FastCVaR function takes in a distribution (d) and a vector of risk (Alpha)
function Fast_CVaR(d,Alpha)
    # Set lambda 
    Lambda = 1 .- Alpha
    
    M = length(Alpha)
    N = nrow(d)
    output = zeros(M)
    j = 1
    
    # Here we precompute repeatedly used values, Psum and XTP.
    d.Psum = cumsum(d.p)
    d.XTP = cumsum(d.X .* d.p)
    
    # Special Case 1: if risk is 0 just use minimum
    while (j <= M) && (Lambda[j] == 0)
        output[j] = minimum(d[d.p .> 0,:].X)
        j+=1
    end  
    
    # General Case for i==1, d[i-1,:] is not valid
    while (j <= M) && (Lambda[j] <= d.Psum[1])
        output[j] = (d.X[1]*(Lambda[j]))/Lambda[j]
        j+=1
    end
    # General Cases
    for i in 2:N
        while (j <= M) && (Lambda[j] <= d.Psum[i])
            output[j] = (d.XTP[i-1] + d.X[i]*(Lambda[j] - d.Psum[i-1]))/Lambda[j]
            j+=1
        end
    end  
    
    while (j <= M)
        output[j] = d.XTP[N]
        j+=1
    end
    
    return output
end

Fast_CVaR (generic function with 1 method)

In [12]:
CVaR_fast = Fast_CVaR(d,Alpha)
t_fast = @elapsed Fast_CVaR(d,Alpha)

append!(DiffNeat, DataFrame(Alg = ["fast"],time = [maximum(abs.(CVaR_fast .- CVaR_neat))]))
append!(TimeDf, DataFrame(Alg = ["fast"],time = [t_fast]))

Row,Alg,time
,String,Float64
1,ori,0.0014564
2,search,0.001073
3,neat,0.0042119
4,decom,0.766608
5,fast,0.0008883


The original CVaR_Vec is $O(MN)$-time while the Fast_CVaR is $O(M+N)$-time.

### Slightly more time consuming test

In [13]:
TimeDf = DataFrame(Alg = ["ori";"search";"neat";"FastT"])
for N in 10 .^ [1;2;3;4] #;5
    for M = 10 .^ [1;2;3;4] #;5
        d2 = DataFrame(X = LinRange(1,10,N), p = 1/N )
        Alpha = LinRange(1,0,M)

        CVaR_X = CVaR_Vec(d2,Alpha)
        t_ori = @elapsed CVaR_Vec(d2,Alpha)
        
        CVaR_search = search_CVaR_Vec(d2,Alpha)
        t_search = @elapsed search_CVaR_Vec(d2,Alpha)
        
        CVaR_neat = neat_CVaR_Vec(d2,Alpha)
        t_neat = @elapsed neat_CVaR_Vec(d2,Alpha)

        CVaR_fast = Fast_CVaR(d2,Alpha)
        t_fast = @elapsed Fast_CVaR(d2,Alpha)

        
        TimeDf[!,string(N)*"d-A"*string(M)] = [t_ori;t_search; t_neat; t_fast]
        
    end
end
TimeDf

Row,Alg,10d-A10,10d-A100,10d-A1000,10d-A10000,100d-A10,100d-A100,100d-A1000,100d-A10000,1000d-A10,1000d-A100,1000d-A1000,1000d-A10000,10000d-A10,10000d-A100,10000d-A1000,10000d-A10000
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,ori,8.46e-5,0.0004345,0.0027605,0.0291149,0.00026,0.00293,0.0247021,0.283871,0.0019236,0.0239949,0.262596,2.54552,0.0215854,0.349144,2.77252,27.7634
2,search,7.72e-5,0.0001436,0.0008773,0.0086789,2.77e-5,0.0001613,0.0010917,0.0113639,4.13e-5,0.000184,0.0010858,0.0383549,0.0006494,0.0007501,0.0014448,0.0128809
3,neat,0.0001202,0.0005967,0.0041401,0.0441435,0.0001522,0.0007889,0.0070909,0.0668231,0.0010829,0.0037618,0.0275989,0.298015,0.0117816,0.0311499,0.236618,1.99325
4,FastT,2.73e-5,0.000124,0.0008295,0.0097037,4.55e-5,0.0001781,0.0010285,0.0080446,0.0003491,0.0003762,0.001125,0.0082921,0.0025018,0.0023476,0.0028994,0.0111897


We can see that the CVaRfastT consistently beating all other primal CVaR method when computing large number of $M$. When $N = 10,000$ and $M=10,000$ it computes in $10,000$ CVaR risk in $<0.003$ seconds. 